In [32]:
import tensorflow as tf
from keras.layers import Dropout
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import adam_v2
import matplotlib.pyplot as plt
import ssl

In [ ]:
batch_size = 32
img_height = 120
img_width = 120

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "recaptcha_dataset/Large",
  validation_split=0.15,
  subset="training",
  seed=1230,
  color_mode="rgb",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical')

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "recaptcha_dataset/Large",
  validation_split=0.15,
  subset="validation",
  seed=1230,
  color_mode="rgb",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical')


In [34]:
class_names = train_ds.class_names

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [36]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
ssl._create_default_https_context = ssl._create_unverified_context
base_model = VGG16(weights='imagenet', include_top=False)


x = base_model.output
x = GlobalAveragePooling2D()(x)


x = Dense(1024, activation='relu')(x)

x = Dropout(0.3)(x) 

predictions = Dense(12, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [37]:
for layer in base_model.layers:
    layer.trainable = False

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_ds,
  validation_data=val_ds,
  epochs=3)

In [39]:
freeze_index = 15

for layer in model.layers[:freeze_index]:
   layer.trainable = False
for layer in model.layers[freeze_index:]:
   layer.trainable = True

In [40]:
model.compile(optimizer=adam_v2.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,
  validation_data=val_ds,
  epochs=5)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()